In [1]:
from langchain import hub

In [6]:
LANGCHAIN_API_KEY='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'

In [7]:
hub.pull("owais/wildfloc-prompt-id")

PromptTemplate(input_variables=['context', 'question'], template='Answer the question based on the context below. If you can\'t answer the question, reply "I don\'t know, ar you sure the question was about wildfloc?".\nContext: {context} \nQuestion: {question}')